<a href="https://colab.research.google.com/github/mohzary/python-deep-learning-f19/blob/master/M2ICP5/M2ICP5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
# To upload data file from local folder to colab
data_file = files.upload()


Saving spam.csv to spam.csv


In [0]:
data = pd.read_csv(io.StringIO(data_file['spam.csv'].decode('latin-1')))


In [5]:
data.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
# Keeping only the neccessary columns
data = data[['v1','v2']]

# to return the lowercased string from the given string. It converts all uppercase characters to lowercase. If no uppercase characters exist, it returns the original string.
data['v2'] = data['v2'].apply(lambda x: x.lower())

data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')


In [7]:
data.head(5)

,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)

In [0]:
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

In [0]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
#print(model.summary())

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)


In [17]:
from keras.callbacks import TensorBoard
tbCallBack= TensorBoard(log_dir='./GraphICP5-3', histogram_freq=0,write_graph=True, write_images=True) 
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 10, batch_size=batch_size, verbose = 2, callbacks=[tbCallBack])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 3733 samples, validate on 1839 samples







Epoch 1/10
 - 33s - loss: 0.1756 - acc: 0.9405 - val_loss: 0.0680 - val_acc: 0.9799

Epoch 2/10
 - 32s - loss: 0.0411 - acc: 0.9871 - val_loss: 0.0665 - val_acc: 0.9821
Epoch 3/10
 - 32s - loss: 0.0203 - acc: 0.9944 - val_loss: 0.0744 - val_acc: 0.9831
Epoch 4/10
 - 32s - loss: 0.0120 - acc: 0.9968 - val_loss: 0.0787 - val_acc: 0.9842
Epoch 5/10
 - 33s - loss: 0.0063 - acc: 0.9984 - val_loss: 0.0873 - val_acc: 0.9821
Epoch 6/10
 - 33s - loss: 0.0056 - acc: 0.9981 - val_loss: 0.0926 - val_acc: 0.9799
Epoch 7/10
 - 33s - loss: 0.0029 - acc: 0.9989 - val_loss: 0.1012 - val_acc: 0.9777
Epoch 8/10
 - 34s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.1154 - val_acc: 0.9821
Epoch 9/10
 - 34s - loss: 0.0010 - acc: 0.9997 - val_loss: 0.1305 - val_acc: 0.9837
Epoch 10/10
 - 33s - loss: 6.6080e-04 - acc: 1.0000 - val_loss: 0.1362 - val_acc: 0.9804


In [18]:
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print("Loss Score: ", (score))
print("Accuracy: %.2f%%" % (acc*100))
print(model.metrics_names)

Loss Score:  0.13618675515200876
Accuracy: 98.04%
['loss', 'acc']
